In [1]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel, AdamW, BertTokenizer, BertForSequenceClassification, BertModel, get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
import torch.nn as nn
import numpy as np
from tqdm import trange
from torch.nn import CrossEntropyLoss
import copy
import torch.optim as optim
import transformers

Using TensorFlow backend.


In [2]:
directory = "../data/glue_data/QQP/"

In [3]:
train = pd.read_csv(directory+"train.tsv", sep = "\t",  quoting = 3)
test = pd.read_csv(directory+"test.tsv", sep = "\t",   quoting = 3)
dev = pd.read_csv(directory+"dev.tsv", sep = "\t",   quoting = 3)

In [4]:
train = train.dropna()
dev = dev.dropna()

In [5]:
model_name = "BERT"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
if model_name == "BERT":
    train_sentences_a = train['question1'].values
    train_sentences_b = train['question2'].values
    train_labels = train['is_duplicate'].values
    
    test_sentences_a = dev['question1'].values
    test_sentences_b = dev['question2'].values
    test_labels = dev['is_duplicate'].values
    
elif model_name == "gpt2":
    train_sentences_a = train['question1'].values
    train_sentences_b = train['question2'].values
    train_labels = train['is_duplicate'].values
    
    test_sentences_a = dev['question1'].values
    test_sentences_b = dev['question2'].values
    test_labels = dev['is_duplicate'].values

In [8]:
max_length = 128
if model_name == "BERT":
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
elif model_name == "gpt2":
    special_tokens_dict = {'bos_token':'_start_','sep_token':'[SEP]', 'cls_token': '[CLS]'}
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2', do_lower_case=True)
    tokenizer.add_special_tokens(special_tokens_dict)

In [9]:
train_input_ids = [tokenizer.encode(sent_a, text_pair=sent_b, add_special_tokens = True) for sent_a, sent_b in zip(train_sentences_a,train_sentences_b)]
test_input_ids = [tokenizer.encode(sent_a, text_pair=sent_b, add_special_tokens = True) for sent_a, sent_b in zip(test_sentences_a,test_sentences_b)]

In [10]:
MAX_LEN = 128
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [11]:
train_attention_masks = []
test_attention_masks = []

for seq in train_input_ids:
    seq_mask = [float(i>0) for i in seq]
    train_attention_masks.append(seq_mask)
    
for seq in test_input_ids:
    seq_mask = [float(i>0) for i in seq]
    test_attention_masks.append(seq_mask)

In [12]:
train_inputs = torch.tensor(train_input_ids)
train_masks = torch.tensor(train_attention_masks)
train_labels = torch.tensor(train_labels).long()

test_inputs = torch.tensor(test_input_ids)
test_masks = torch.tensor(test_attention_masks)
test_labels = torch.tensor(test_labels).long()

In [13]:
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [14]:
if model_name == "BERT":
    model = BertForSequenceClassification.from_pretrained("/tmp/BERT/MRPC", num_labels=2)
    #model = torch.load('./COLA-finetuned-BERT-v2.pt')
elif model_name == "gpt2":
    #model = GPT2DoubleHeadsModel.from_pretrained("gpt2", num_labels = 2)
    model = torch.load('./MRPC-finetuned-GPT2.pt')
    model.resize_token_embeddings(len(tokenizer))
model = model.cuda()

In [18]:
linears = [nn.Linear(768, 2).to(torch.device("cuda:0")) for  x in range(0,13)]

In [19]:
param_optimizer = [list(linears[i].named_parameters()) for i in range(0,13)]
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [[
    {'params': [p for n, p in param_optimizer[i] if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer[i] if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
] for i in range(0,13)]

optimizer = [AdamW(optimizer_grouped_parameters[i],
                     lr=2e-5) for i in range(0,13)]

scheduler = [get_linear_schedule_with_warmup(optimizer[i], num_warmup_steps=0, num_training_steps=len(train_dataloader)*3) for i in range (0,13)]

In [20]:
if model_name=="BERT":
  model.bert.config.output_hidden_states = True
  model.bert.config.is_decoder = False
  model.bert.encoder.output_hidden_states = True
  for i in range(0,len(model.bert.encoder.layer)): 
    model.bert.encoder.layer[i].is_decoder = False
    model.bert.encoder.layer[i].output_hidden_states = True
else:
    model.transformer.output_hidden_states = True

In [21]:
# Store our loss and accuracy for plotting
train_loss_sets = [[] for i in range(0,13)]

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  for linear in linears:
    linear.train()
    
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    #b_input_ids, b_masks, b_labels, b_cls_positions = batch
    b_input_ids, b_masks, b_labels = batch
    for i in range(0,13):
      optimizer[i].zero_grad()
    if(model_name=="gpt2"):
        #model.transformer.output_hidden_states = True
        outputs = model.transformer(b_input_ids, attention_mask=b_masks)
        logits = []
        for i in range(0,13):
            hl = outputs[2][i] ## We taken the all hidden states and take the l layer
            #h0 = outputs[3][0] ## Here we are using embeddings layer 
            #logits = linear(h0.view(-1,98304)) ## We flat the embeddings
            logits.append(linears[i](hl[range(0,len(b_cls_positions)),b_cls_positions])) ## We take the classification token embedding to train the linear layer
    else:
        #h0 = model.bert.embeddings(b_input_ids)
        #logits = linear(h0.view(-1,98304))
        outputs = model.bert(b_input_ids, attention_mask=b_masks)
        logits = []
        for i in range(0,13):
            hl = outputs[2][i] ## We taken the all hidden states and take the l layer
            #h0 = outputs[3][0] ## Here we are using embeddings layer 
            #logits = linear(h0.view(-1,98304)) ## We flat the embeddings
            logits.append(linears[i](hl[:,0])) ## We take the first token [CLS] embedding to train the linear layer

    for i in range(0,13): 
      loss_fct = CrossEntropyLoss()
      loss=loss_fct(logits[i].view(-1, logits[i].size(-1)),
                              b_labels.view(-1))
      
      train_loss_sets[i].append(loss.item())
      loss.backward(retain_graph=True)
      optimizer[i].step()
      # Update learning rate schedule
      scheduler[i].step()
    
#if model == "gpt2":
#  torch.save(linear, "/content/drive/My Drive/linear_gpt2_layer1.pt")
#else:
#  torch.save(linear,"/content/drive/My Drive/linear_BERT.pt")

Epoch: 100%|██████████| 3/3 [12:39:04<00:00, 15181.35s/it] 


In [22]:
preds = [[] for i in range(0,13)]

for linear in linears:
    linear.eval()

for step, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    #b_input_ids, b_masks, b_labels, b_cls_positions = batch
    b_input_ids, b_masks, b_labels = batch
    with torch.no_grad():
        if(model_name=="gpt2"):
            outputs = model.transformer(b_input_ids, attention_mask=b_masks)
            logits = []
            for i in range(0,13):
                hl = outputs[2][i] ## We taken the all hidden states and take the l layer
                #h0 = outputs[3][0] ## Here we are using embeddings layer 
                #logits = linear(h0.view(-1,98304)) ## We flat the embeddings
                logits.append(linears[i](hl[range(0,len(b_cls_positions)),b_cls_positions])) ## We take the classification token embedding to t
        else:
            outputs = outputs = model.bert(b_input_ids, attention_mask=b_masks)
            logits = []
            for j in range(0,13):
                hl = outputs[2][j] ## We taken the all hidden states and take the l layer
                #h0 = outputs[3][0] ## Here we are using embeddings layer 
                #logits = linear(h0.view(-1,98304)) ## We flat the embeddings
                logits.append(linears[j](hl[:,0])) ## We take the first token [CLS] embedding to train the linear layer
    for j in range(0,13):
        logits[j] = logits[j].detach().cpu().numpy()
        preds[j].append(logits[j])

In [23]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
for i in range(0,13):
    print("layer " + str(i))
    predictions = []
    for pred in preds[i]:
        p = np.argmax(pred, axis = 1)
        for label in p:
            predictions.append(label)
    print(classification_report(test_labels, predictions, digits = 4))
    print("Accuracy: ", accuracy_score(test_labels, predictions))

layer 0
              precision    recall  f1-score   support

           0     0.6318    1.0000    0.7744     25545
           1     0.0000    0.0000    0.0000     14885

    accuracy                         0.6318     40430
   macro avg     0.3159    0.5000    0.3872     40430
weighted avg     0.3992    0.6318    0.4893     40430

Accuracy:  0.6318327974276527
layer 1
              precision    recall  f1-score   support

           0     0.6652    0.9026    0.7660     25545
           1     0.5688    0.2204    0.3177     14885

    accuracy                         0.6515     40430
   macro avg     0.6170    0.5615    0.5418     40430
weighted avg     0.6297    0.6515    0.6009     40430

Accuracy:  0.651471679445956
layer 2
              precision    recall  f1-score   support

           0     0.6791    0.8927    0.7714     25545
           1     0.5999    0.2760    0.3781     14885

    accuracy                         0.6657     40430
   macro avg     0.6395    0.5844    0.5748  

/home/test/anaconda3/envs/transformers-v2.5.1/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy:  0.6656937917388078
layer 3
              precision    recall  f1-score   support

           0     0.6978    0.8763    0.7769     25545
           1     0.6216    0.3489    0.4469     14885

    accuracy                         0.6821     40430
   macro avg     0.6597    0.6126    0.6119     40430
weighted avg     0.6698    0.6821    0.6554     40430

Accuracy:  0.6820925055651744
layer 4
              precision    recall  f1-score   support

           0     0.7401    0.8418    0.7876     25545
           1     0.6446    0.4926    0.5584     14885

    accuracy                         0.7132     40430
   macro avg     0.6923    0.6672    0.6730     40430
weighted avg     0.7049    0.7132    0.7033     40430

Accuracy:  0.7132080138511007
layer 5
              precision    recall  f1-score   support

           0     0.7530    0.8315    0.7903     25545
           1     0.6478    0.5319    0.5842     14885

    accuracy                         0.7212     40430
   macro avg  